In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
import os, sys
import itertools
import datetime

import pandas as pd

# To load personal libraries (currently in folder 'tools'):
sys.path.append('tools')

#import generic_tools_pl as gt   # like hexcolor(), ...
#import specific_nn_tools_pl as nnt  # like CustomDataset(), Net(), train_model() 
import generic_tools_pl as gt
from generic_tools_pl import region

gt.region= input("Enter the region number: ")
region=gt.region
print(gt.region)

48


In [2]:
def save_data_file(data, filename, data_dir='.', label='data', rewrite=False, df=False):
                   
    current_filename = os.path.join(data_dir,filename)

    if (os.path.isfile(current_filename) and rewrite) or not os.path.isfile(current_filename) :
        if os.path.isfile(current_filename) :
            print(f" Saving current {label} to filename '{filename}' ... ** File exists: REWRITED **")
        else:
            print(f" Saving current {label} to filename '{filename}'")
        if df :
            data.to_pickle(os.path.join(data_dir,filename) )
        else:
            pickle.dump(data, open( os.path.join(data_dir,filename), "wb" ))
    else:
        if os.path.isfile(current_filename) and not rewrite :
            print(f" ** Warning: {label} file '{filename}' already exists, saving is not possible **\n ** in directory '{data_dir}'\n ** You can authorize rewriting saying True to 'rewrite' option.\n")

def get_combi_ganh(combi_gan, hist_isim) :
    '''
    Ajoute aux combinaisons des indices des trois forcages GAN (premier argument) les indices du forcage HIST (deuxième argument)
    en faisant des combinaisons entre les deux.
    
    Retourne la nouvelle liste des combinaisons GANH.
    '''
    import itertools

    # Combinaisons GAN + H
    combi_ganh_raw = list(itertools.product(combi_gan,hist_isim))

    # Combinaisons GANH (concatenne la sous-liste [GAN] avec H)
    combi_ganh = [ [*gan,h] for gan,h in combi_ganh_raw]

    return combi_ganh

def add_h_to_combi_gan(combi_gan, hist_isim) :
    '''
    Ajoute aux combinaisons des indices des trois forcages GAN (premier argument) les indices du forcage HIST (deuxième argument) choisies aleatoirement.
    Il n'y a pas des combinaisons entre les deux.  Un element de la liste COMBI_GAN d'entree ne sera associé qu'à un seul indice de HIST.
    
    Retourne la nouvelle liste des combinaisons GANH ayant le même nombre d'elements que COMBI_GAN d'entree.
    '''
    import itertools

    i_hist_isim = hist_isim[0]     # indice du debut
    n_hist_isim = len(hist_isim)   # nombre de profils HIST
    n_combi_gan = len(combi_gan)   # nombre de combinaisons G-A-N

    if n_hist_isim <= n_combi_gan:
        i_hist_permuted = (np.random.permutation(n_combi_gan) % n_hist_isim).tolist()
    else:
        # si nombre de combinaisons GAN inferieur au nombre de profils HIST
        # ... alors on prend n_combi_gan indices de HISY aleatoirement et
        #     on les associe aux cominaisons GAN existantes
        i_hist_permuted = np.random.permutation(n_hist_isim).tolist()
        i_hist_permuted = i_hist_permuted[:n_combi_gan]
    
    # Combinaisons GANH (concatenne la sous-liste [GAN] avec H)
    combi_ganh = [ [*gan,i_hist_isim+ihist] for gan,ihist in zip(combi_gan,i_hist_permuted)]

    return combi_ganh

In [3]:
# Call set_printoptions(threshold=x) to change the threshold where arrays are truncated. To prevent truncation, set the threshold to np.inf.
# Change linewidth to print wider lines ...
#display(np.get_printoptions())
if True:
    np.set_printoptions(threshold=np.inf,linewidth=180) # good for a 1920 pixels wide screen
    pd.set_option('display.max_columns', 18)            #
else:
    np.set_printoptions(threshold=np.inf,linewidth=240)   # good for an iMac 27" screen
    pd.set_option('display.max_columns', 24)              #

#pd.set_option('display.max_columns', 100)             # (this is maybe too much!)

In [4]:
# Label identifiant la méthode de génération des données
#data_gener_method = 'TTDGM1'  # Répartition GAN et Combinaisons HIST avec GAN Random
data_gener_method = 'TTDGM2'  # Répartition GAN et Combinaisons HIST avec GAN sans repetition

seed_for_combi = 0    # "semence" d'initialisation du générateur de chiffres aléatoires
train_part = 0.85     # part pour Repartition GAN en TRAIN, le reste en TEST

# nombre de données (combinaisons GANH) tirées par modèle, exacte pour TRAIN, maximum pour TEST (decommenter le choix)
#n_sim_for_train_by_mod = 132;   n_sim_max_for_test_by_mod = 100
n_sim_for_train_by_mod = 1000;  n_sim_max_for_test_by_mod = 1000

In [5]:
# -----------------------------------------------------------------------------------------------
# Selection of working Forcing and Models:
# -----------------------------------------------------------------------------------------------

# list of selected forcings
forcing_names = [ 'ghg', 'aer', 'nat', 'hist' ]
forcing_color = [ 'r',   'b',   'g',   'k' ]

# list of selected models 
model_names = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']
#model_names = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MRI-ESM2-0', 'NorESM2-LM']

In [6]:
# ether_dir = '/net/ether/data/varclim/ggalod/Constantin'

#data_dir = '/datatmp/data/constantin_data/data_source'
#data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source'
#data_dir = 'data'

# On lit le dataset de train
# Repertoire des donnees
try:
    # WORK dir carlos projet ryn sur Jean Zay
    data_dir = "/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region{}".format(region)
    if not os.path.isdir(data_dir):
        print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # WORK dir Guillaume sur Jean Zay ** NON **
        #data_dir = '/gpfswork/rech/ryn/rces866/Constantin'
        #if not os.path.isdir(data_dir):
        #print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # SSD sur Acratopotes au Locean
        data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source_pl'
        if not os.path.isdir(data_dir):
            print(f" ** data_dir '{data_dir}' not found. Trying next...")
            
            # sur Cloud SU (carlos)
            data_dir = os.path.expanduser('~/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source_pl')
            if not os.path.isdir(data_dir):
                print(f" ** data_dir '{data_dir}' not found. Trying next...")
                
                # en dernier recours, en esperant qu'il y a un repertoire 'data' present ...
                data_dir = os.path.expanduser('data')
                if not os.path.isdir(data_dir):
                    print(f" ** data_dir '{data_dir}' not found at all **\n")
                    raise Exception('data_dir not found')

except Exception as e:
    print(f'\n *** Exception error "{e}" ***\n')
    raise

print(f"data_dir found at '{data_dir}'")

data_dir found at '/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region48'


In [7]:
save_data_ok = True
#save_data_ok = False

#--------------------
#save_anc_data_only_ok = True  # if TRUE saves a dictionary with index, number by mod, list of models, of forcings, of years. but not data arrays
save_anc_data_only_ok = False
#--------------------
dataframe_format_ok = True
array_format_ok = False
#--------------------
#rewrite_files_ok = True
rewrite_files_ok = False
#--------------------
# verbose = False
verbose = True
#--------------------
save_figs = True
#save_figs = False
#--------------------
figs_dir = 'Figs'

local_fig_label = "Nb0_gener_TT_data_set_to_PL_df"   # label a appliquer aux noms des fichiers (figures) produits dans le notebook

figs_defaults = { 'dpi' : 300, 'facecolor' : 'w', 'edgecolor' : 'w'} # ajuter format='png' ou autre a l'appel de savefig()

if save_figs and not os.path.exists(figs_dir):
    os.makedirs(figs_dir)

In [8]:
data_dic = gt.load_basic_data_and_gener_df(source_dirname=data_dir,
                                           models=model_names,
                                           sim_file="All_sim_z{}v0.p".format(region), 
                                           models_file="models_z{}v0.p".format(region),
                                           forcings=forcing_names,
                                           data_dir=data_dir, 
                                           verbose=True)

years = data_dic['years']
T_ghg_df, T_aer_df, T_nat_df, T_hist_df = data_dic['list_of_df']


Lecture des meta donnees (ALL_sim, liste de modeles, ...) ...

Data label: '{' '.join(sim_name_splitted)} ({data_indent_version})'

 List of 12 models:
   ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']

 List of 5 forcings:
   ['ghg', 'aer', 'nat', 'hist', 'other']

 'All_sim_z48v0.p' contents [forcings x models]:
[[ 3  3  3  8 50  3 10  4  8  3  5  3]
 [ 3  3  2  8 30  3 10  4  8 10  5  3]
 [ 3  3  3  8 50  3 20  4  8 50  5  3]
 [15  3 11 28 65  6 28  5 31 50 11  3]]
 Dim: (4, 12)

 Sum by column (number of simulations by model):
 [ 24  12  19  52 195  15  68  17  55 113  26  12]

 Sum by line (number of simulations by forcing):
 [103  89 160 256]

 INI and END indices for data tables in 'T_xxx_z48v0.p' files (for each forcing 'xxx'):
 INI index:
[[  0   3   6   9  17  67  70  80  84  92  95 100]
 [  0   3   6   8  16  46  49  59  63  71  81  86]
 [  0   3   6   9 

In [9]:
for df in data_dic['list_of_df']:
    print(df.shape)

(103, 166)
(89, 166)
(160, 166)
(256, 166)


In [10]:
data_dic

{'label': "{' '.join(sim_name_splitted)} ({data_indent_version})",
 'data_dir': '/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region48',
 'models': ['ACCESS-ESM1-5',
  'BCC-CSM2-MR',
  'CESM2',
  'CNRM-CM6-1',
  'CanESM5',
  'FGOALS-g3',
  'GISS-E2-1-G',
  'HadGEM3-GC31-LL',
  'IPSL-CM6A-LR',
  'MIROC6',
  'MRI-ESM2-0',
  'NorESM2-LM'],
 'forcings': ['ghg', 'aer', 'nat', 'hist'],
 'forcing_color_dic': {'ghg': '#b22222',
  'aer': '#000080',
  'nat': '#808000',
  'hist': '#000000'},
 'forcing_color_names_dic': {'ghg': 'FireBrick',
  'aer': 'Navy',
  'nat': 'Olive',
  'hist': 'Black'},
 'forcing_inv_color_dic': {'ghg': '#ff0000',
  'aer': '#0000FF',
  'nat': '#008000',
  'hist': '#d2691e'},
 'forcing_inv_color_names_dic': {'ghg': 'Red',
  'aer': 'Blue',
  'nat': 'Green',
  'hist': 'Chocolate'},
 'forcing_light_color_dic': {'ghg': '#f08080',
  'aer': '#00bfff',
  'nat': '#9acd32',
  'hist': '#ff8c00'},
 'forcing_light_color_names_dic': {'ghg': 'LightCoral',
  'aer': 

In [11]:
T_hist_df.iloc[:,:2].groupby('model').count().values.flatten(),[int(v) for v in np.ceil(T_hist_df.iloc[:,:2].groupby('model').count().values.flatten()*.15).tolist()]

(array([15,  3, 11, 28, 65,  6, 28,  5, 31, 50, 11,  3]),
 [3, 1, 2, 5, 10, 1, 5, 1, 5, 8, 2, 1])

In [12]:
# Liste de simulations par modele:
if verbose:
    T_ilist_by_mod = []
    for imod,mod in enumerate(model_names) :
        print(f'{imod:2d}) {mod:15s} ',end='')

        # liste de simulations par forcage du modele en cours:
        T_mod_isim_by_forcing = []
        for iforcing,forcing in enumerate(forcing_names) :
            current_t_df_varname = f'T_{forcing.lower()}_df'
            xxx_df = globals()[current_t_df_varname]
            xxc_indices = xxx_df.loc[lambda df: df['model'] == mod].index
            ii = xxc_indices[0]    # indice inferieur (incluyant)
            jj = xxc_indices[-1]+1 # indice superieur (excluyant)
            #if verbose:
            #    print('\n  ',iforcing, forcing, ii, jj, jj-ii,end='')

            # cumule toutes les simulations du modele et forcage en cours
            T_mod_isim_by_forcing.append(np.arange(ii,jj).tolist())

        # cumule la liste de simulations de chaque forcage pour le modele en cours
        T_ilist_by_mod.append(T_mod_isim_by_forcing)

        # pour affichage:
        T_shape_by_forcing = [len(x) for x in T_mod_isim_by_forcing]
        T_shape_by_forcing_no_hist = [len(x) for x in T_mod_isim_by_forcing[:-1]]
        #print(T_shape_by_forcing)
        #print(T_shape_by_forcing_no_hist)

        print(f" ... [{', '.join([str(d) for d in T_shape_by_forcing]):15s}]", end='')
        print(f' {np.prod([n1 for n1 in T_shape_by_forcing_no_hist]):6} comb.GAN', end='')
        print(f' {np.prod([n1 for n1 in T_shape_by_forcing]):8} comb.GANH')

 0) ACCESS-ESM1-5    ... [3, 3, 3, 15    ]     27 comb.GAN      405 comb.GANH
 1) BCC-CSM2-MR      ... [3, 3, 3, 3     ]     27 comb.GAN       81 comb.GANH
 2) CESM2            ... [3, 2, 3, 11    ]     18 comb.GAN      198 comb.GANH
 3) CNRM-CM6-1       ... [8, 8, 8, 28    ]    512 comb.GAN    14336 comb.GANH
 4) CanESM5          ... [50, 30, 50, 65 ]  75000 comb.GAN  4875000 comb.GANH
 5) FGOALS-g3        ... [3, 3, 3, 6     ]     27 comb.GAN      162 comb.GANH
 6) GISS-E2-1-G      ... [10, 10, 20, 28 ]   2000 comb.GAN    56000 comb.GANH
 7) HadGEM3-GC31-LL  ... [4, 4, 4, 5     ]     64 comb.GAN      320 comb.GANH
 8) IPSL-CM6A-LR     ... [8, 8, 8, 31    ]    512 comb.GAN    15872 comb.GANH
 9) MIROC6           ... [3, 10, 50, 50  ]   1500 comb.GAN    75000 comb.GANH
10) MRI-ESM2-0       ... [5, 5, 5, 11    ]    125 comb.GAN     1375 comb.GANH
11) NorESM2-LM       ... [3, 3, 3, 3     ]     27 comb.GAN       81 comb.GANH


In [13]:
%%time

#verbose = False

# Initialisation des listes retournées par la boucle ---------------------------------------------------------------------
train_combi_list = []   # liste des indices pour composer l'ensemble TRAIN. Les indices font reference aux lignes des tableaux lus dans fichiers "T_{fff}_v4.p"
test_combi_list  = []   # liste des indices pour composer l'ensemble TEST. 
#
n_train_combi_list = [] # liste du nombre de données en TRAIN par modèle (une répétition du même nombre normalement selon la variable n_sim_for_train_by_mod)
n_test_combi_list  = [] # liste du nombre de données en TEST par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_for_test_by_mod)
# ------------------------------------------------------------------------------------------------------------------------

for imod,mod in enumerate(model_names) :
    #imod = 0
    #imod = 4
    #mod = model_names[imod]

    if verbose :
        print(f"\n{'-'*92}",end='')
    print(f'\n{imod:2d}) {mod:15s} ',end='')

    # initialise le generateur de nombres aléatoires
    print(f"\n Re-initialise le generateur de nombres aleatoires par le SEED: {seed_for_combi}")
    np.random.seed(seed_for_combi)

    # liste de simulations par forcage du modele en cours:
    T_mod_isim_by_forcing = []
    for iforcing,forcing in enumerate(forcing_names) :
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]
        xxc_indices = xxx_df.loc[lambda df: df['model'] == mod].index
        ii = xxc_indices[0]    # indice inferieur (incluyant)
        jj = xxc_indices[-1]+1 # indice superieur (excluyant)
        if verbose:
            print('\n  ',iforcing, forcing, ii, jj, jj-ii,end='')

        # cumule toutes les simulations du modele et forcage en cours
        T_mod_isim_by_forcing.append(np.arange(ii,jj).tolist())
        
    if verbose :
        print(f'\n  Indices des forcages GANH (GHG, AER, NAT, GIST):')
        [print(f"\n    {len(ksim)} indices pour {forcing_names[ii].upper()}: {ksim}",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
        #print(f'\n  {mod}: indices des forcages GAN (GHG, AER, NAT):')
        #[print(f"    - {len(T_mod_isim_by_forcing[jf])} indices des simulations {ff.upper()}: {T_mod_isim_by_forcing[jf]}") for jf,ff in enumerate(forcing_names[:-1])]
    else:
        print(f'\n  Indices des forcages ',end='')
        [print(f"  {len(ksim)} pour {forcing_names[ii].upper()},",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
        print('')

    all_combi_gan = list(itertools.product(*T_mod_isim_by_forcing[:-1]))
    n_combi_gan = len(all_combi_gan)
    
    if verbose :
        print(f'\n  {n_combi_gan} combinaisons GAN:')
        print('  ',*all_combi_gan[:9], ' ...\n    ', *all_combi_gan[-9:])

    permuted_combi_gan = np.random.permutation(all_combi_gan).tolist()
    if verbose :
        print(f'\n  {n_combi_gan} combinaisons GAN permutees:')
        print('  ',*permuted_combi_gan[:9], ' ...\n    ', *permuted_combi_gan[-9:])

    # repartition des combinaisons de simulations GAN en TRAIN et TEST
    n_gan_train = int(n_combi_gan * train_part)
    n_gan_test = n_combi_gan - n_gan_train

    train_combi_gan = permuted_combi_gan[:n_gan_train]
    test_combi_gan = permuted_combi_gan[n_gan_train:]

    if verbose :
        print(f'\n  {n_combi_gan} combinaisons GAN reparties selon:')
        print(f'   - {train_part*100:.1f}% des combinaisons GAN pour TRAIN: {n_gan_train}')
        print(f'   - {(1-train_part)*100:.1f}% combinaisons GAN pour TEST: {n_gan_test}')

    # -----------------------------------------------------------
    # Methode TTDGM1: Répartition GAN et Combinaisons HIST avec GAN Random

    if data_gener_method == 'TTDGM1' :
        # Methode TTDGM1: Répartition GAN et Combinaisons HIST avec GAN Random
        hist_isim = T_mod_isim_by_forcing[-1]  # indices des HIST

        # Combinaisons GAN + H
        train_combi_ganh = get_combi_ganh(train_combi_gan, hist_isim)
        test_combi_ganh = get_combi_ganh(test_combi_gan, hist_isim)
        
    elif data_gener_method == 'TTDGM2' :
        # Methode TTDGM2: Répartition GAN et Combinaisons HIST avec GAN sans repetition
        hist_isim = T_mod_isim_by_forcing[-1]   # indices des HIST
        
        # Combinaisons GAN + H
        train_combi_ganh = add_h_to_combi_gan(train_combi_gan, hist_isim)
        test_combi_ganh = add_h_to_combi_gan(test_combi_gan, hist_isim)

    n_train_combi_ganh = len(train_combi_ganh)
    n_test_combi_ganh = len(test_combi_ganh)
    
    permuted_train_combi_ganh = np.random.permutation(train_combi_ganh).tolist()

    if verbose :
        print(f'\n  TRAIN: {n_train_combi_ganh} combinaisons GANH:')
        print('  ',*train_combi_ganh[:7], ' ...\n    ', *train_combi_ganh[-7:])

        print(f'\n  TEST: {n_test_combi_ganh} combinaisons GANH:')
        print('  ',*test_combi_ganh[:7], ' ...\n    ', *test_combi_ganh[-7:])

        print(f'\n  TRAIN: {n_train_combi_ganh} combinaisons GANH permutees:')
        print('  ',*permuted_train_combi_ganh[:7], ' ...\n    ', *permuted_train_combi_ganh[-7:])

    # ------------------------------------------------------------------------------------------------------------------------
    # TRAIN Set:
    # Nombre fixe de patterns (combinaisons G-A-N-H) par modèle. Combinaisons choisies aleatoirement.
    # Permet repetitions (si le nombre de combinaisons max est insuffissant pour combler le numero de
    # patterns requis).
    # ------------------------------------------------------------------------------------------------------------------------

    # si le nombre de patterns est inferieur au requis alors on multiplie les donnees (duplique, x3 ...) autant que necessaire 
    if n_train_combi_ganh < n_sim_for_train_by_mod :
        n_times_combi_ganh = int(np.ceil(n_sim_for_train_by_mod / n_train_combi_ganh))
        permuted_train_combi_ganh = permuted_train_combi_ganh * n_times_combi_ganh
        
        if verbose :
            print(f"\n  permuted_train_combi_ganh is too small ({n_train_combi_ganh} sim. combinations). Smaller than spected number of train patterns ({n_sim_for_train_by_mod}) for model.",end='')
        print(f"  ** Multiplying {n_times_combi_ganh} times the size of permuted_train_combi_ganh to have enough data to select for training.")

    print(f"  - Selecting only {n_sim_for_train_by_mod} / {len(permuted_train_combi_ganh)} sim. combinations from for TRAIN set for model.")

    train_combi_list += permuted_train_combi_ganh[:n_sim_for_train_by_mod]
    n_train_combi_list += [n_sim_for_train_by_mod]

    # ------------------------------------------------------------------------------------------------------------------------
    # TEST Set:
    # Nombre variable de patterns (combinaisons G-A-N-H) par modèle. Pas de repetitions. Toutes les combinaisons G-A-N-H sont
    # prise en compte mais avec un nombre max limite. Combinaisons choisies aleatoirement uniquement dans le cas ou il faut
    # restreindre leur nombre.
    # ------------------------------------------------------------------------------------------------------------------------

    if len(test_combi_ganh) > n_sim_max_for_test_by_mod :
        if verbose :
            print(f"\n  Number of TEST sim. combinations ({n_test_combi_ganh}) is too high. Restricted to only {n_sim_max_for_test_by_mod} patterns (randomlly).")
        permuted_test_combi_ganh = np.random.permutation(test_combi_ganh).tolist()
        test_combi_ganh = permuted_test_combi_ganh[:n_sim_max_for_test_by_mod]
    
        print(f" ** Restricting to {n_sim_max_for_test_by_mod} / {n_test_combi_ganh} sim. combinations for TEST set for model.")
    else:
        print(f"  - Selecting all {len(test_combi_ganh)} sim. combinations for TEST set for model.")
    
    test_combi_list += test_combi_ganh
    n_test_combi_list += [len(test_combi_ganh)]

# conversion en array d'indices les listes
train_combi_index = np.array(train_combi_list)  # indices pour composer l'ensemble TRAIN. Par colonnes, les indices font reference aux lignes des tableaux lus dans fichiers "T_{fff}_v4.p"
test_combi_index = np.array(test_combi_list)    # indices pour composer l'ensemble TEST. 
#
n_train_combi = n_train_combi_list     # nombre de données en TRAIN par modèle (une repetition du même nombre normalement selon la variable n_sim_for_train_by_mod)
n_test_combi = n_test_combi_list       # nombre de données en TEST par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_for_test_by_mod)

#### _Sauvegarde des tableaux d'indices et du nombre de simulations par modèle pour les ensembles de TRAIN et TEST_


--------------------------------------------------------------------------------------------
 0) ACCESS-ESM1-5   
 Re-initialise le generateur de nombres aleatoires par le SEED: 0

   0 ghg 0 3 3
   1 aer 0 3 3
   2 nat 0 3 3
   3 hist 0 15 15
  Indices des forcages GANH (GHG, AER, NAT, GIST):

    3 indices pour GHG: [0, 1, 2]
    3 indices pour AER: [0, 1, 2]
    3 indices pour NAT: [0, 1, 2]
    15 indices pour HIST: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
  27 combinaisons GAN:
   (0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 2, 0) (0, 2, 1) (0, 2, 2)  ...
     (2, 0, 0) (2, 0, 1) (2, 0, 2) (2, 1, 0) (2, 1, 1) (2, 1, 2) (2, 2, 0) (2, 2, 1) (2, 2, 2)

  27 combinaisons GAN permutees:
   [0, 0, 2] [2, 2, 0] [1, 1, 2] [1, 2, 2] [0, 1, 2] [1, 0, 2] [2, 1, 2] [1, 1, 1] [2, 0, 1]  ...
     [2, 0, 0] [1, 0, 0] [0, 2, 1] [2, 1, 1] [0, 1, 0] [0, 0, 0] [2, 1, 0] [1, 2, 0] [1, 1, 0]

  27 combinaisons GAN reparties selon:
   - 85.0% des combinaisons GAN pour TRAI


  75000 combinaisons GAN permutees:
   [63, 33, 37] [62, 21, 35] [41, 30, 66] [27, 42, 41] [32, 27, 52] [36, 20, 22] [49, 34, 39] [61, 27, 32] [54, 19, 36]  ...
     [43, 26, 29] [49, 28, 17] [53, 36, 43] [44, 45, 60] [31, 20, 60] [47, 33, 58] [45, 28, 30] [46, 17, 34] [62, 31, 35]

  75000 combinaisons GAN reparties selon:
   - 85.0% des combinaisons GAN pour TRAIN: 63750
   - 15.0% combinaisons GAN pour TEST: 11250

  TRAIN: 63750 combinaisons GANH:
   [63, 33, 37, 70] [62, 21, 35, 104] [41, 30, 66, 118] [27, 42, 41, 107] [32, 27, 52, 63] [36, 20, 22, 90] [49, 34, 39, 99]  ...
     [39, 30, 27, 59] [62, 24, 22, 71] [39, 39, 38, 79] [35, 39, 39, 74] [52, 20, 43, 77] [41, 35, 59, 119] [25, 22, 43, 106]

  TEST: 11250 combinaisons GANH:
   [62, 35, 19, 59] [46, 43, 26, 66] [29, 25, 65, 105] [65, 44, 39, 85] [47, 34, 35, 77] [59, 41, 56, 67] [20, 21, 45, 80]  ...
     [53, 36, 43, 80] [44, 45, 60, 94] [31, 20, 60, 77] [47, 33, 58, 60] [45, 28, 30, 102] [46, 17, 34, 62] [62, 31, 35, 95]


In [14]:
print(f"\nEcriture de la structure des tableaux de forcages, sous forme de dictionaire, pour TRAIN et pour TEST:\n"+\
      f" - tableaux d'indices,\n"+\
      f" - liste du nombre de forcages par modèle,\n"+\
      f" - liste de modeles presents,\n"+\
      f" - liste de forcages presents:\n")

if save_data_ok :
    
    train_set_suffix = f"{data_gener_method}-S{seed_for_combi}-GAN{train_part*100:.0f}pc-N{n_sim_for_train_by_mod}_z{region}v0"
    test_set_suffix = f"{data_gener_method}-S{seed_for_combi}-GAN{(1-train_part)*100:.0f}pc-NMx{n_sim_max_for_test_by_mod}_z{region}v0"

    current_train_dic = { 'index' : train_combi_index,
                          'n_by_mod' : n_train_combi,
                          'models' : model_names,
                          'forcings' : forcing_names,
                          'years' : years }
    current_train_combi_dic_filename = f"train-combi-dic_{train_set_suffix}.p"
    save_data_file(current_train_dic, current_train_combi_dic_filename, data_dir=data_dir,
                   label='train combi dictionary', rewrite=rewrite_files_ok)

    current_test_dic = { 'index' : test_combi_index,
                         'n_by_mod' : n_test_combi,
                         'models' : model_names,
                         'forcings' : forcing_names,
                         'years' : years  }
    current_test_combi_dic_filename = f"test-combi-dic_{test_set_suffix}.p"
    save_data_file(current_test_dic, current_test_combi_dic_filename, data_dir=data_dir,
                   label='test combi dictionary', rewrite=rewrite_files_ok)
    
else:
    print(f"\n ** data saving not authorized: set True to `save_data_ok` and Boolean variable **\n")


Ecriture de la structure des tableaux de forcages, sous forme de dictionaire, pour TRAIN et pour TEST:
 - tableaux d'indices,
 - liste du nombre de forcages par modèle,
 - liste de modeles presents,
 - liste de forcages presents:

 Saving current train combi dictionary to filename 'train-combi-dic_TTDGM2-S0-GAN85pc-N1000_z48v0.p'
 Saving current test combi dictionary to filename 'test-combi-dic_TTDGM2-S0-GAN15pc-NMx1000_z48v0.p'


In [15]:
if data_gener_method == 'TTDGM1' :
    train_combi_index_bd1 = train_combi_index.copy()
    test_combi_index_bd1 = test_combi_index.copy()

    current_train_dic_bd1 = current_train_dic.copy()
    current_test_dic_bd1 = current_test_dic.copy()
    print('bd1',train_combi_index_bd1.shape,test_combi_index_bd1.shape)

if data_gener_method == 'TTDGM2' :
    train_combi_index_bd2 = train_combi_index.copy()
    test_combi_index_bd2 = test_combi_index.copy()

    current_train_dic_bd2 = current_train_dic.copy()
    current_test_dic_bd2 = current_test_dic.copy()
    print('bd2',train_combi_index_bd2.shape,test_combi_index_bd2.shape)

bd2 (12000, 4) (1731, 4)


In [16]:
save_data_ok , save_anc_data_only_ok, rewrite_files_ok

(True, False, False)

In [17]:
print(f"\nEcriture des donnees des forcages:\n")

if save_data_ok and (True or not save_anc_data_only_ok):
    
    for iforcing,forcing in enumerate(forcing_names) :
        print(f"\n{forcing.upper()}:")
    
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]

        current_train_index = train_combi_index[:,iforcing]
        current_test_index = test_combi_index[:,iforcing]

        if dataframe_format_ok :
            current_train_filename = f"train-{forcing.lower()}_{train_set_suffix}_df.p"
            current_test_filename = f"test-{forcing.lower()}_{test_set_suffix}_df.p"

            train_df = xxx_df.loc[current_train_index]
            train_df.reset_index(inplace=True, drop=True)

            save_data_file(train_df, current_train_filename, data_dir=data_dir,
                           label=f'train {forcing.upper()} forcing df', df=True,
                           rewrite=rewrite_files_ok)

            test_df = xxx_df.loc[current_test_index]
            test_df.reset_index(inplace=True, drop=True)

            save_data_file(test_df, current_test_filename, data_dir=data_dir,
                           label=f'test {forcing.upper()} forcing df', df=True,
                           rewrite=rewrite_files_ok)

        if array_format_ok :
            current_train_filename = f"train-{forcing.lower()}_{train_set_suffix}.p"
            current_test_filename = f"test-{forcing.lower()}_{test_set_suffix}.p"

            train_array = xxx_df.loc[current_train_index].drop('model', axis=1).values
            save_data_file(train_array, current_train_filename, data_dir=data_dir,
                           label=f'train {forcing.upper()} forcing array', rewrite=rewrite_files_ok)

            test_array = xxx_df.loc[current_test_index].drop('model', axis=1).values
            save_data_file(test_array, current_test_filename, data_dir=data_dir,
                           label=f'test {forcing.upper()} forcing array', rewrite=rewrite_files_ok)

            print(f" train {forcing.upper()} array -> dim: {train_array.shape}")
            print(f" test {forcing.upper()} array -> dim: {test_array.shape}")

elif save_data_ok and save_anc_data_only_ok:
    print(f" ** saving ancillary data only, data arrays not allowed to be saved. To allow saving, set False to `save_anc_data_only_ok` Boolean variable **\n")

else:
    print(f" ** data saving not allowed. To allow saving, set True to `save_data_ok` and False to `save_anc_data_only_ok` Boolean variables **\n")


Ecriture des donnees des forcages:


GHG:
 Saving current train GHG forcing df to filename 'train-ghg_TTDGM2-S0-GAN85pc-N1000_z48v0_df.p'


 Saving current test GHG forcing df to filename 'test-ghg_TTDGM2-S0-GAN15pc-NMx1000_z48v0_df.p'

AER:
 Saving current train AER forcing df to filename 'train-aer_TTDGM2-S0-GAN85pc-N1000_z48v0_df.p'
 Saving current test AER forcing df to filename 'test-aer_TTDGM2-S0-GAN15pc-NMx1000_z48v0_df.p'

NAT:
 Saving current train NAT forcing df to filename 'train-nat_TTDGM2-S0-GAN85pc-N1000_z48v0_df.p'
 Saving current test NAT forcing df to filename 'test-nat_TTDGM2-S0-GAN15pc-NMx1000_z48v0_df.p'

HIST:
 Saving current train HIST forcing df to filename 'train-hist_TTDGM2-S0-GAN85pc-N1000_z48v0_df.p'
 Saving current test HIST forcing df to filename 'test-hist_TTDGM2-S0-GAN15pc-NMx1000_z48v0_df.p'


In [18]:
data_dir

'/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region48'

In [19]:
print(f"\nFIN d'Execution:\n  case_out_dir ......... {train_set_suffix} et {test_set_suffix}\n  Date/heure d'arret ... {datetime.datetime.now()}")


FIN d'Execution:
  case_out_dir ......... TTDGM2-S0-GAN85pc-N1000_z48v0 et TTDGM2-S0-GAN15pc-NMx1000_z48v0
  Date/heure d'arret ... 2023-07-06 16:55:33.139549
